In [74]:
import pandas as pd
df=pd.read_csv('https://raw.githubusercontent.com/megelon/datasets/master/datos.csv')
df['order_demand']=df.order_demand.astype(float).fillna(0)
df.head(1)

In [122]:
min(list(set(df.order_demand)))

0.0

In [105]:
df.to_csv("/home/megelon/madgit/datasets/data.csv", index=False) 

In [106]:
df1=pd.read_csv("/home/megelon/madgit/datasets/data.csv")

In [107]:
df['order_demand']=df.order_demand.astype(float)

In [108]:
df1.dtypes

product_code         object
warehouse            object
product_category     object
date                 object
order_demand        float64
dtype: object

In [109]:
df["date"] = pd.to_datetime(df.date)

In [110]:
df.dtypes

product_code                object
warehouse                   object
product_category            object
date                datetime64[ns]
order_demand               float64
dtype: object

# creacion de variables

In [142]:
dft=df[['date','order_demand']].sort_values(by='date').dropna()

In [143]:
len(list(set(list(dft.date))))

1729

In [144]:
dft=dft.set_index('date')
dft.head(2)

,order_demand
date,
2011-01-08,2.0
2011-05-31,108.0


In [145]:
dft['dow']=dft.index.dayofweek

In [146]:
dft.head(2)

,order_demand,dow
date,,
2011-01-08,2.0,5
2011-05-31,108.0,1


In [147]:
dft.dtypes

order_demand    float64
dow               int64
dtype: object

# kmeans

In [ ]:
from sklearn import metrics
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans

clusters=range(2,15)
meandist=[]

ch=[]
ss=[]
l_k=[]

for k in clusters:
    model=KMeans(n_clusters=k)
    model.fit(dft)
    labels=model.predict(dft)
    meandist.append(sum(np.min(cdist(dft, model.cluster_centers_, 'euclidean'), axis=1))
    / dft.shape[0])
    
    ch.append(metrics.calinski_harabaz_score(dft, labels))
    ss.append(metrics.silhouette_score(dft, labels, metric='euclidean'))
    l_k.append(k)

df_criterios=pd.DataFrame(data=meandist,columns=['meandist'])
df_criterios['k']=l_k
df_criterios['calinski_harabas']=ch
df_criterios['silhouette']=ss    

#Codo: Distancia euclideana mínima media
y = meandist
z = l_k
n = meandist

fig, ax = plt.subplots()
ax.set_title('Diagrama de Codo', fontdict={'fontsize': 20}, loc='center', pad=None)
ax.plot(z, y, 'orange')
ax.set_ylabel('Distancia euclideana mínima media', color='black')
#ax.tick_params('y', colors='g')

for i, txt in enumerate(n):
    ax.annotate(int(txt), (z[i], y[i]))

#candinski
y = ch
z = l_k
n = ch

fig, ax = plt.subplots()
ax.set_title('Criterio de parada', fontdict={'fontsize': 20}, loc='center', pad=None)
ax.plot(z, y, 'g--')
ax.set_ylabel('Calinski Harabas', color='g')
#ax.tick_params('y', colors='g')

for i, txt in enumerate(n):
    ax.annotate(int(txt), (z[i], y[i]))

y = ss
z = l_k
n = ss

ax2 = ax.twinx()
s2 = ss
ax2.plot(z, s2,'b--')
ax2.set_ylabel('Silhouette', color='b')
#ax2.tick_params('y', colors='b')

for i, txt in enumerate(n):
    ax2.annotate(round(txt,2), (z[i], y[i]))

/home/megelon/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function calinski_harabaz_score is deprecated; Function 'calinski_harabaz_score' has been renamed to 'calinski_harabasz_score' and will be removed in version 0.23.
  warnings.warn(msg, category=FutureWarning)


# Hierarchical

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
from sklearn import metrics

l_method=[]
l_metric=[]
l_evaluate=[]

for method in ['single','complete','average','weighted']:
    for metric in ['cityblock', 'correlation', 'cosine', 'euclidean']:
        link_matrix = linkage(dft, method=method,metric=metric)
        c, coph_dists = cophenet(link_matrix, pdist(dft))
        l_evaluate.append(c)
        l_method.append(method)
        l_metric.append(metric)

df_evaluate=pd.DataFrame(data=l_method, columns=['metodo'])
df_evaluate['metrica']=l_metric
df_evaluate['dist_cofenetica']=l_evaluate
df_evaluate=df_evaluate.sort_values(by='dist_cofenetica', ascending=False)

method=df_evaluate.iloc[0,0]
metric=df_evaluate.iloc[0,1]

link_matrix = linkage(dft, method=method,metric=metric)

print(method + " : " + metric)
df_evaluate.head()